# Script for web scrapping cwc real time data
### import libraries 

In [5]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import datetime
from datetime import date
from datetime import timedelta
import matplotlib.pyplot as plt
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
import os

### date range and html json responses

###### monsoon data retrieval will take time as it will extract all station hourly data

In [6]:
# Get today's date
today = date.today()
# Yesterday date
yesterday = today - timedelta(days = 1)
day_bef_yesterday=today - timedelta(days = 2)
day_bef_bef_yes=today - timedelta(days = 3)
d1 = yesterday.strftime("%Y-%m-%d")
d1_under=yesterday.strftime("%Y_%m_%d")
d2=day_bef_yesterday.strftime("%Y-%m-%d")
d2_under=day_bef_yesterday.strftime("%Y_%m_%d")
d3= day_bef_bef_yes.strftime("%Y-%m-%d")
d3_under=day_bef_bef_yes.strftime("%Y_%m_%d")
# d3_under
# print(d1)
# print(d2)
# print(d3)

#### yesterdays data

In [7]:
sdate1=d2 
edate1=d1
# sdate1="2023-08-22"    #yyyy-mm-dd
# edate1="2023-08-22"    #yyyy-mm-dd # may(5) 31  sep(9) 30


url1=f"""https://ffs.india-water.gov.in/iam/api/new-entry-data/specification/sorted?sort-criteria=%7B%22sortOrderDtos%22:%5B%7B%22sortDirection%22:%22ASC%22,%22field%22:%22id.dataTime%22%7D%5D%7D&specification=%7B%22where%22:%7B%22where%22:%7B%22expression%22:%7B%22valueIsRelationField%22:false,%22fieldName%22:%22id.datatypeCode%22,%22operator%22:%22eq%22,%22value%22:%22HHS%22%7D%7D,%22and%22:%7B%22expression%22:%7B%22valueIsRelationField%22:false,%22fieldName%22:%22dataValue%22,%22operator%22:%22null%22,%22value%22:%22false%22%7D%7D%7D,%22and%22:%7B%22expression%22:%7B%22valueIsRelationField%22:false,%22fieldName%22:%22id.dataTime%22,%22operator%22:%22btn%22,%22value%22:%22{sdate1}T00:00:00.250,{edate1}T23:59:59.250%22%7D%7D%7D"""

md1=None
# session1=requests.session()
# data1=session1.get(url1)
# soup1=BeautifulSoup(data1.content, 'html.parser')

# try:
#     md1=json.loads(soup1.get_text())
# except:
#     md1=json.loads(soup1.get_text())

session1=requests.session()
# data1=session1.get(url1,timeout=1000)
# soup1=BeautifulSoup(data1.content, 'html.parser')

# try:
#     md1=json.loads(soup1.get_text())
# except:
#     md1=json.loads(soup1.get_text())
    
try:
    data1 = session1.get(url1, timeout=1000)
    soup1 = BeautifulSoup(data1.content, 'html.parser')
    md1 = json.loads(soup1.get_text())
except requests.exceptions.ConnectionError as e:
    print(f"ConnectionError: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

In [9]:
# md1

### updating data in database
##### here yesterdays data will be added to existing retrived data base

In [10]:
new=0 # this is just for checking number of new updates
old=0 # this is just to count not updated data
# df_code=pd.read_excel('name-code.xlsx',index_col=0)
df_code=pd.read_excel('F:\\phd_work\\cwc_real_time_data\\updated_real_time\\data_files\\2020_updated\\name-code.xlsx',index_col=0)
updated_path = "F:\\phd_work\\cwc_real_time_data\\updated_real_time\\data_files\\2020_updated\\data"
for name_x in df_code['name']:
    print(name_x)
    df1=df_code[df_code['name']==name_x] # remove this line if want to download all data
    dic=df1.T.to_dict()
    fl=[]


    for j in md1[:]:
        try:

            ml=[j['id']['stationCode'],datetime.datetime.fromisoformat(j['id']['dataTime']),dic[j['id']['stationCode']]['name'],j['dataValue']]
            fl.append(ml)
        except:
            pass

    try:
        excel_file=updated_path+'\\'+name_x+'2020_to'+d2_under+'.xlsx'
        excel_file_new=updated_path+'\\'+name_x+'2020_to'+d1_under+'.xlsx'
        fl=sorted(fl)   
        fl_size_half=round(len(fl)/2)-1
        df=pd.DataFrame(fl)
        df.columns=['code','time','name','WSE']
        df=df.loc[fl_size_half:] #round(len(fl)/2)-1 = 23 after monsoon we will not get 23 data so here we have to update
#         df=df.loc[fl_size_half:] #round(len(fl)/2)-1 = 23 after monsoon we will not get 23 data so here we have to update
#         df=df.loc[47:]

    #         df.to_excel(file_path_exp+name_x+'2023_june_July.xlsx') # change file name here
        # opening existing excel and appending updated data 
        sheet_name = 'Sheet1'
        workbook = openpyxl.load_workbook(excel_file)
        sheet = workbook[sheet_name]

        # Determine the starting row for appending
        start_row = sheet.max_row + 1

        # Append new data to the sheet
        last_index = sheet.max_row - 1  # Subtract 1 to account for 0-based indexing
#         df.index = range(last_index + 1, last_index + 1 + len(df))  # Adjust the index
        df.index = range(last_index, last_index + len(df))  # Adjust the index
        # Append new data to the sheet
        for row in dataframe_to_rows(df, index=True, header=False):
            sheet.append(row)

        # Save the changes
        workbook.save(excel_file)
        workbook.close()
        workbook=None
        workbook
        os.rename(excel_file, excel_file_new)
        new=new+1
        print('updated file count:::==>>> '+str(new)+' <<< updated path >>>>>'+excel_file_new)


    except:
        try:
            excel_file=updated_path+'\\'+name_x+'2020_to'+d3_under+'.xlsx'
            excel_file_new=updated_path+'\\'+name_x+'2020_to'+d1_under+'.xlsx'
            fl=sorted(fl)   
            df=pd.DataFrame(fl)
            df.columns=['code','time','name','WSE']
#             df=df.loc[23:]

        #         df.to_excel(file_path_exp+name_x+'2023_june_July.xlsx') # change file name here
            # opening existing excel and appending updated data 
            sheet_name = 'Sheet1'
            workbook = openpyxl.load_workbook(excel_file)
            sheet = workbook[sheet_name]

            # Determine the starting row for appending
            start_row = sheet.max_row + 1

            # Append new data to the sheet
            last_index = sheet.max_row - 1  # Subtract 1 to account for 0-based indexing
    #         df.index = range(last_index + 1, last_index + 1 + len(df))  # Adjust the index
            df.index = range(last_index, last_index + len(df))  # Adjust the index
            # Append new data to the sheet
            for row in dataframe_to_rows(df, index=True, header=False):
                sheet.append(row)

            # Save the changes
            workbook.save(excel_file)
            workbook.close()
            workbook=None
            workbook
            os.rename(excel_file, excel_file_new)
            new=new+1
            print('updated file count:::==>>> '+str(new)+' <<< updated path >>>>>'+excel_file_new)
            
        except:
            try:
                
                
                excel_file=updated_path+'\\'+name_x+'2020_to'+d3_under+'.xlsx'
                excel_file_new=updated_path+'\\'+name_x+'2020_to'+d2_under+'.xlsx'
                os.rename(excel_file, excel_file_new)
                old=old+1
                print('No new data updated for ::'+name_x+' <<< non updated count >>>>'+str(old))
                print('File is not updated but renamed with the day before date')

            except:
                print('File is not found and not updated>>>>>> '+name_x+' <<<<<<<')

Parwan pick-up Weir
File is not found and not updated>>>>>> Parwan pick-up Weir <<<<<<<
Kalisindh Dam
File is not found and not updated>>>>>> Kalisindh Dam <<<<<<<
Mirawadi
File is not found and not updated>>>>>> Mirawadi <<<<<<<
Yadgir
updated file count:::==>>> 1 <<< updated path >>>>>F:\phd_work\cwc_real_time_data\updated_real_time\data_files\2020_updated\data\Yadgir2020_to2023_10_28.xlsx
Wadakbal
File is not found and not updated>>>>>> Wadakbal <<<<<<<
Ithai_WLR
File is not found and not updated>>>>>> Ithai_WLR <<<<<<<
Panchana Dam
File is not found and not updated>>>>>> Panchana Dam <<<<<<<
Targaon
updated file count:::==>>> 2 <<< updated path >>>>>F:\phd_work\cwc_real_time_data\updated_real_time\data_files\2020_updated\data\Targaon2020_to2023_10_28.xlsx
Kursela
updated file count:::==>>> 3 <<< updated path >>>>>F:\phd_work\cwc_real_time_data\updated_real_time\data_files\2020_updated\data\Kursela2020_to2023_10_28.xlsx
VAPI
updated file count:::==>>> 4 <<< updated path >>>>>F:\phd_

In [36]:
# df_code=pd.read_excel('name-code.xlsx',index_col=0)
# df1=df_code[df_code['name']=='Tikarpara'] # remove this line if want to download all data
# dic=df1.T.to_dict()
# fl=[]
# for j in md1[:]:
#     try:

#         ml=[j['id']['stationCode'],datetime.datetime.fromisoformat(j['id']['dataTime']),dic[j['id']['stationCode']]['name'],j['dataValue']]
#         fl.append(ml)
#     except:
#         pass
# fl
# # updated_path = "F:\\phd_work\\cwc_real_time_data\\updated_real_time\\data_files\\2020_updated\\data"
# # excel_file=updated_path+'\\'+name_x+'2020_to'+d2_under+'.xlsx'
# # excel_file_new=updated_path+'\\'+name_x+'2020_to'+d1_under+'.xlsx'
# fl=sorted(fl)   
# df=pd.DataFrame(fl)
# # df.columns=['code','time','name','WSE']
# # df=df.loc[23:]
# round(len(fl)/2)-1